# CIFAR-10 using Residual Neural Networks

## Importing Libraries and Packages

In [1]:
import numpy as np 
import pandas as pd 
from tensorflow import keras
from keras import Model, Input
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Add, Activation
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

## Loading and Preparing Data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Before moving on let's normalize the imgae data so that it is between 0 and 1.

In [3]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

As the labels are intergers let's convert them into array for simpler processing.

In [5]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

## Defining functions for Convolution blocks and Building Model 

In [6]:
def convolution_block(X, filters, f = 3):
    X_shortcut = X
    F1, F2 = filters 
    
    # For the convolutional part
    X = Conv2D(filters = F1  , kernel_size=(f,f), activation = "relu", padding = "Same")(X)
    X = Conv2D(filters = F2, kernel_size=(f,f), activation = "relu", padding = "Same")(X)

    # For the shortcut path
    X_shortcut = Conv2D(filters = F2, kernel_size=(1,1), activation = "relu", padding = "Same")(X_shortcut)

    #
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)
    
    return X

In [7]:
def res_net(input_shape = (32,32,3), classes = 10) :
    X_input = Input(input_shape)

    X = Conv2D(8, kernel_size = (3,3), activation = 'relu', padding = "Same")(X_input)
    X = BatchNormalization()(X)
    
    #Convolution Block 1
    X = convolution_block(X, filters = [8,16] , f = 5)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)
    X = BatchNormalization()(X)
    
    #Convolution Block 2
    X = convolution_block(X, filters = [16,32] , f = 5)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)
    X = BatchNormalization()(X)
    
    #Convolution Block 3
    X = convolution_block(X, filters = [32,63] , f = 3)
    X = MaxPooling2D(pool_size=(2,2), strides =2)(X)
    X = BatchNormalization()(X)

    #Convolution Block 4
    X = Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = "Same")(X)
    X = BatchNormalization()(X)
    
    X = Flatten()(X)
    X = Dense(200, activation = 'relu')(X)
    X = BatchNormalization()(X)
    X_output = Dense(10, activation = 'softmax')(X)
    
    model = Model(X_input, X_output)
    
    return model

Let's name the model and see it's structure and other important architectural details.

In [8]:
model = res_net(input_shape = (32,32,3), classes = 10)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 8)    224         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 8)    1608        batch_normalization[0][0]        
_______________________________________________________________________________________

## Training and testing the model
This code below defines the important hyperparameters for training of model.<br>
We're using learning rate decay for smoother and accurate training of model.

In [9]:
batch_size = 64
epochs = 20
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=5000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

Compiling and fitting the model.

In [10]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer , metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.0)

Epoch 1/20
782/782 [==============================] - 224s 287ms/step - loss: 1.4015 - accuracy: 0.5015
Epoch 2/20
782/782 [==============================] - 218s 279ms/step - loss: 0.9226 - accuracy: 0.6733
Epoch 3/20
782/782 [==============================] - 234s 299ms/step - loss: 0.7424 - accuracy: 0.7390
Epoch 4/20
782/782 [==============================] - 256s 328ms/step - loss: 0.6380 - accuracy: 0.7781
Epoch 5/20
782/782 [==============================] - 241s 308ms/step - loss: 0.5571 - accuracy: 0.8060
Epoch 6/20
782/782 [==============================] - 243s 311ms/step - loss: 0.4837 - accuracy: 0.8297
Epoch 7/20
782/782 [==============================] - 242s 310ms/step - loss: 0.4136 - accuracy: 0.8551
Epoch 8/20
782/782 [==============================] - 275s 352ms/step - loss: 0.3545 - accuracy: 0.8753
Epoch 9/20
782/782 [==============================] - 305s 390ms/step - loss: 0.2907 - accuracy: 0.8967
Epoch 10/20
782/782 [==============================] - 320s 410m

Checking the accuracy of the model.

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.2895078659057617
Test accuracy: 0.7775999903678894
